In [71]:
class Pmf():
    
    """Probability Mass Function（確率質量関数）"""
    
    def __init__(self):
        self.hypos = []
        self.priors = {}
        self.priors_given_likelihoods = {}
        self.posteriors = {}
    
    def Set(self, x, prob):
        self.hypos.append(x)
        self.priors[x] = prob
    
    def Values(self):
        return self.hypos

    def Incr(self, x, i):
        self.priors[x] = self.priors[x] + 1
        
    def Mult(self, x, likelihood):
        self.priors_given_likelihoods[x] = self.priors[x] * likelihood
        
    def Normalize(self):
        for x in self.priors_given_likelihoods.keys():
            self.posteriors[x] = self.priors_given_likelihoods[x] / (sum(self.priors_given_likelihoods.values()))
            self.priors[x] = self.posteriors[x] # TODO 正規化後は、事後分布が次の事前分布になる

    def Items(self):
        return self.posteriors.items()
            
    def Prob(self, x):
        return self.posteriors[x]

In [72]:
class Suite(Pmf):
    
    def __init__(self, hypos):
        Pmf.__init__(self) # TODO ここの書き方がよくわからない。Mix-inみたいなの？
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        raise UnimplementedMethodException()
        
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
        
    def Print(self):
        """Prints the hypotheses and their probabilities."""
        for hypo, prob in sorted(self.Items()):
             print("{}: {}".format(hypo, prob))

In [73]:
class Dice(Suite):
    def Likelihood(self, data, hypo):
        if hypo < data:
            return 0
        else:
            return 1.0 / hypo

In [74]:
suite = Dice([4, 6, 8, 12, 20])

suite.priors

suite.Update(6)
print('After one 6')
suite.Print()

for roll in [6, 8, 7, 7, 5, 4]:
    suite.Update(roll)

print("")
print('After more rolls')
suite.Print()

After one 6
4: 0.0
6: 0.3921568627450981
8: 0.2941176470588236
12: 0.19607843137254904
20: 0.11764705882352944

After more rolls
4: 0.0
6: 0.0
8: 0.9432484536722127
12: 0.055206128061290875
20: 0.0015454182664965536
